In [7]:
import pandas as pd
from scipy import stats
from scipy.stats import mannwhitneyu
# 데이터 전처리 과정 - 기존 데이터에서 수정사항 발생
df = pd.read_csv("./추출단어정리.csv")
df['word'] = df['word'].replace('mysql', 'sql')
df['word'] = df['word'].replace('db', 'sql')
df['word'] = df['word'].replace('excel', '엑셀')
df = df.groupby("word",as_index = False).agg({"count":"sum"})
df = df.sort_values(by="count",ascending = False)
df=df[:10]

df_original = pd.read_csv("./Inflearn_crawling.csv")
df_original['강의명'] = df_original['강의명'].str.lower() #강의명 모두 소문자화
def convert_to_int(x):
    # 숫자만 추출
    num_str = ''.join(filter(str.isdigit, str(x)))
    # 추출된 문자열이 비어 있지 않으면 숫자로 변환, 그렇지 않으면 0 반환
    return int(num_str) if num_str else 0

df_original['강의 금액'] = df_original['강의 금액'].apply(convert_to_int)
df_original['수강생 수'] = df_original['수강생 수'].apply(convert_to_int)

# 총 매출 계산
df_original['총 매출'] = df_original['강의 금액'] * df_original['수강생 수']

# 매출만 남기기
df_original = df_original[['강의명','총 매출']]

# 각 DataFrame을 저장할 리스트 생성
df_with_keyword_list = []

# 데이터 프레임으로 저장
for keyword in df['word']:
    df_filtered = df_original[df_original['강의명'].str.contains(keyword, case=False)]
    df_with_keyword_list.append((f'df_{keyword}', df_filtered))

# df_키워드 입력하면 해당 데이터 추출됨
# 변수로 저장
for key, value in df_with_keyword_list:
    globals()[key] = value
    
# 새로운 데이터프레임을 저장할 딕셔너리
df_without_keyword = {}

# 각 키워드에 대해 새로운 데이터프레임 생성 및 해당 키워드가 들어간 행 제거
for keyword in df['word']:
    df_without_keyword[keyword] = df_original[~df_original['강의명'].str.contains(keyword)].reset_index(drop=True)
    # 개별 데이터프레임을 변수에 저장
    globals()[f"df_non_{keyword}"] = df_without_keyword[keyword]
    
# Shapiro-Wilk 정규성 검정과 등분산 검정
for keyword in df['word']:
    stat1, p_value1 = stats.shapiro(globals()[f"df_{keyword}"]['총 매출'])
    stat2, p_value2 = stats.shapiro(globals()[f"df_non_{keyword}"]['총 매출'])
    bartlett_stat, bartlett_p_value = stats.bartlett(globals()[f"df_{keyword}"]['총 매출'], globals()[f"df_non_{keyword}"]['총 매출'])
    levene_stat, levene_p_value = stats.levene(globals()[f"df_{keyword}"]['총 매출'], globals()[f"df_non_{keyword}"]['총 매출'])

    # 결과 출력
    print(f"키워드 '{keyword}'에 대한 Shapiro-Wilk 정규성 검정 p-value: {p_value1}")
    print(f"키워드 '{keyword}'가 없는 그룹에 대한 Shapiro-Wilk 정규성 검정 p-value: {p_value2}")
    print(f"키워드 '{keyword}'에 대한 Bartlett 등분산 검정 p-value: {bartlett_p_value}")
    print(f"키워드 '{keyword}'가 없는 그룹에 대한 Levene 등분산 검정 p-value: {levene_p_value}")
    print('----------------------------')    
    

키워드 '데이터'에 대한 Shapiro-Wilk 정규성 검정 p-value: 1.9432009137565228e-26
키워드 '데이터'가 없는 그룹에 대한 Shapiro-Wilk 정규성 검정 p-value: 1.0859459811088494e-29
키워드 '데이터'에 대한 Bartlett 등분산 검정 p-value: 5.1185707984321944e-05
키워드 '데이터'가 없는 그룹에 대한 Levene 등분산 검정 p-value: 0.716568586270674
----------------------------
키워드 '파이썬'에 대한 Shapiro-Wilk 정규성 검정 p-value: 7.543331410897469e-12
키워드 '파이썬'가 없는 그룹에 대한 Shapiro-Wilk 정규성 검정 p-value: 8.035374890154209e-36
키워드 '파이썬'에 대한 Bartlett 등분산 검정 p-value: 3.3865349416974432e-09
키워드 '파이썬'가 없는 그룹에 대한 Levene 등분산 검정 p-value: 0.9405866135371033
----------------------------
키워드 '분석'에 대한 Shapiro-Wilk 정규성 검정 p-value: 5.110379528472137e-16
키워드 '분석'가 없는 그룹에 대한 Shapiro-Wilk 정규성 검정 p-value: 1.4946672401869436e-33
키워드 '분석'에 대한 Bartlett 등분산 검정 p-value: 1.3425071799478786e-43
키워드 '분석'가 없는 그룹에 대한 Levene 등분산 검정 p-value: 0.2804373831807044
----------------------------
키워드 '활용'에 대한 Shapiro-Wilk 정규성 검정 p-value: 4.76919115310892e-10
키워드 '활용'가 없는 그룹에 대한 Shapiro-Wilk 정규성 검정 p-value: 9.617101405836929

In [8]:
for keyword in df['word']:
    # Mann-Whitney U 검정 수행
    statistic, p_value = mannwhitneyu(globals()[f"df_{keyword}"]['총 매출'], globals()[f"df_non_{keyword}"]['총 매출'])

    # 결과 출력
    print(f"키워드 '{keyword}'에 대한 검정 통계량:", statistic)
    print(f"키워드 '{keyword}'에 대한 p-value:", p_value)

    # p-value를 통해 가설 검정
    alpha = 0.05
    if p_value < alpha:
        print("귀무가설을 기각합니다. 두 집단의 중위수는 다를 것으로 판단됩니다.")
    else:
        print("귀무가설을 기각하지 않습니다. 두 집단의 중위수는 같을 것으로 판단됩니다.")
        
    print('----------------------')


키워드 '데이터'에 대한 검정 통계량: 15454.0
키워드 '데이터'에 대한 p-value: 0.01738309214786103
귀무가설을 기각합니다. 두 집단의 중위수는 다를 것으로 판단됩니다.
----------------------
키워드 '파이썬'에 대한 검정 통계량: 10255.5
키워드 '파이썬'에 대한 p-value: 0.00925323670335411
귀무가설을 기각합니다. 두 집단의 중위수는 다를 것으로 판단됩니다.
----------------------
키워드 '분석'에 대한 검정 통계량: 12720.0
키워드 '분석'에 대한 p-value: 0.4434414857049346
귀무가설을 기각하지 않습니다. 두 집단의 중위수는 같을 것으로 판단됩니다.
----------------------
키워드 '활용'에 대한 검정 통계량: 7840.5
키워드 '활용'에 대한 p-value: 0.7613186449250514
귀무가설을 기각하지 않습니다. 두 집단의 중위수는 같을 것으로 판단됩니다.
----------------------
키워드 '기초'에 대한 검정 통계량: 6603.0
키워드 '기초'에 대한 p-value: 0.5933930234325882
귀무가설을 기각하지 않습니다. 두 집단의 중위수는 같을 것으로 판단됩니다.
----------------------
키워드 'sql'에 대한 검정 통계량: 7696.0
키워드 'sql'에 대한 p-value: 0.1406653652072619
귀무가설을 기각하지 않습니다. 두 집단의 중위수는 같을 것으로 판단됩니다.
----------------------
키워드 '입문'에 대한 검정 통계량: 6571.5
키워드 '입문'에 대한 p-value: 0.6282157343656523
귀무가설을 기각하지 않습니다. 두 집단의 중위수는 같을 것으로 판단됩니다.
----------------------
키워드 '개발'에 대한 검정 통계량: 2015.0
키워드 '개발'에 대한 p-value: 0.0010979